In [1]:
import global_align as ga

In [2]:
GAP_SCORE = -2 
scoring_mat = {
    "A": {"A": 1, "C": -1, "G": -1, "T": -1, "-": -2},
    "C": {"A": -1, "C": 1, "G": -1, "T": -1, "-": -2},
    "G": {"A": -1, "C": -1, "G": 1, "T": -1, "-": -2},
    "T": {"A": -1, "C": -1, "G": -1, "T": 1, "-": -2},
    "-": {"A": -2, "C": -2, "G": -2, "T": -2, "-": -2},
}

In [3]:
scoring_mat["C"]["C"]

1

In [4]:
seq_1 = "CTGCG"
seq_2 = "GTCG"

# 0: gap in seq_1
# 1: gap in seq_2
# 2: match/mismatch

In [5]:
m = len(seq_1)
n = len(seq_2)
F_mat_num_rows = m + 1
F_mat_num_cols = n + 1

In [6]:
# https://www.freecodecamp.org/news/list-within-a-list-in-python-initialize-a-nested-list/
partial_F_mat = [
    [0]*(F_mat_num_cols) for i in range(2)
]
partial_F_mat

[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]

We can go one row at a time through the F_mat (starting at row index 1, col index 1 and always skipping col index 0).  Simultaneously, fill out the best_paths_mat.  (Note that we need to save the entirety of the best_paths_mat.)  We only need to keep the col 0 of the F_mat and two rows in memory at a time.

In [7]:
partial_F_mat[0] = [GAP_SCORE*x for x in range(F_mat_num_cols)]
partial_F_mat

[[0, -2, -4, -6, -8], [0, 0, 0, 0, 0]]

In [8]:
partial_F_mat[1][0] = GAP_SCORE

partial_F_mat

[[0, -2, -4, -6, -8], [-2, 0, 0, 0, 0]]

In [9]:
best_paths_mat_num_rows = F_mat_num_rows
best_paths_mat_num_cols = F_mat_num_cols
best_paths_mat = [[0]*best_paths_mat_num_cols for i in range(best_paths_mat_num_rows)]

# Get 1's in the beginning of each row.
for i in range(1, best_paths_mat_num_rows):
    best_paths_mat[i][0] = 1

best_paths_mat

[[0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0]]

In [10]:
# Pre loop
i = 1
partial_F_mat_prev_row_id = 0
partial_F_mat_cur_row_id = 1

for j in range(1, F_mat_num_cols):
    seq_1_index = i - 1
    seq_2_index = j - 1

    score_for_path_a = partial_F_mat[partial_F_mat_cur_row_id][j - 1]
    score_for_path_b = partial_F_mat[partial_F_mat_prev_row_id][j]
    score_for_path_c = partial_F_mat[partial_F_mat_prev_row_id][j - 1]

    score_for_gap_in_seq_1 = score_for_path_a + GAP_SCORE
    score_for_gap_in_seq_2 = score_for_path_b + GAP_SCORE
    score_for_match_mismatch = score_for_path_c + scoring_mat[seq_1[seq_1_index]][seq_2[seq_2_index]]
    
    possible_new_scores = [score_for_gap_in_seq_1, score_for_gap_in_seq_2, score_for_match_mismatch]
    max_possible_new_score = max(possible_new_scores)
    
    partial_F_mat[partial_F_mat_cur_row_id][j] = max_possible_new_score
    best_type_of_path = possible_new_scores.index(max_possible_new_score)
    # print(f"i: {i}      j: {j}")
    best_paths_mat[i][j] = best_type_of_path


In [11]:
partial_F_mat

[[0, -2, -4, -6, -8], [-2, -1, -3, -3, -5]]

In [15]:
for i in range(2, F_mat_num_rows):
    # Prep for a new row iteration.
    # https://stackoverflow.com/a/14836456
    # Do some swapping. 
    partial_F_mat_prev_row_id, partial_F_mat_cur_row_id = partial_F_mat_cur_row_id, partial_F_mat_prev_row_id
    # Update the 0th column based on how gaps are penalized.
    partial_F_mat[partial_F_mat_cur_row_id][0] = GAP_SCORE*i
    
    for j in range(1, F_mat_num_cols):
        seq_1_index = i - 1
        seq_2_index = j - 1

        score_for_path_a = partial_F_mat[partial_F_mat_cur_row_id][j - 1]
        score_for_path_b = partial_F_mat[partial_F_mat_prev_row_id][j]
        score_for_path_c = partial_F_mat[partial_F_mat_prev_row_id][j - 1]

        score_for_gap_in_seq_1 = score_for_path_a + GAP_SCORE
        score_for_gap_in_seq_2 = score_for_path_b + GAP_SCORE
        score_for_match_mismatch = score_for_path_c + scoring_mat[seq_1[seq_1_index]][seq_2[seq_2_index]]
        
        possible_new_scores = [score_for_gap_in_seq_1, score_for_gap_in_seq_2, score_for_match_mismatch]
        max_possible_new_score = max(possible_new_scores)
        
        partial_F_mat[partial_F_mat_cur_row_id][j] = max_possible_new_score
        best_type_of_path = possible_new_scores.index(max_possible_new_score)
        # print(f"i: {i}      j: {j}")
        best_paths_mat[i][j] = best_type_of_path

In [16]:
partial_F_mat

[[-8, -5, -4, -1, -2], [-10, -7, -6, -3, 0]]

In [17]:
best_paths_mat

[[0, 0, 0, 0, 0],
 [1, 2, 0, 2, 0],
 [1, 1, 2, 0, 0],
 [1, 2, 1, 2, 2],
 [1, 1, 1, 2, 2],
 [1, 1, 1, 1, 2]]

In [18]:
# traceback
# Prepare for loop.
seq_1_aligned = []
seq_2_aligned = []

num_alignment_moves = max(best_paths_mat_num_rows, best_paths_mat_num_cols) - 1

# Start at the bottom-right.
seq_1_index = m - 1
seq_2_index = n - 1

for w in range(num_alignment_moves):
    # Prep for this iteration.
    # Because of the initial row and column in
    # best_paths_mat that doesn't align with
    # any parts of the two sequence, the indices
    # are off by one.
    best_paths_mat_row_index = seq_1_index + 1
    best_paths_mat_col_index = seq_2_index + 1

    path_indicator = best_paths_mat[best_paths_mat_row_index][best_paths_mat_col_index]

    if path_indicator == 0:
        seq_1_aligned.append("-")
        seq_2_aligned.append(seq_2[seq_2_index])
        seq_2_index -= 1
    elif path_indicator == 1:
        seq_1_aligned.append(seq_1[seq_1_index])
        seq_1_index -= 1
        seq_2_aligned.append("-")
    else:
        seq_1_aligned.append(seq_1[seq_1_index])
        seq_1_index -= 1
        seq_2_aligned.append(seq_2[seq_2_index])
        seq_2_index -= 1
    

In [19]:
seq_1_aligned.reverse()
seq_2_aligned.reverse()

In [20]:
print(seq_1_aligned)
print(seq_2_aligned)

['C', 'T', 'G', 'C', 'G']
['G', 'T', '-', 'C', 'G']
